This notebooks desribes, in detail, the first data augmentation step: given an extended simulation cell, extract the specified window size (here fixed to 100 pixels, corresponding to 12 Angstrom for the employed resolution of 0.12 Angstrom per pixel).

The pixel/Ansgrom ratio is specified in the following:

In [ ]:
pixel_to_angstrom = 0.12

# Import libraries

In [ ]:
import os
import h5py
import matplotlib.pyplot as plt
import numpy as np
import numpy.matlib
from scipy import ndimage

# Change default figure size

In [ ]:
plt.rcParams['figure.figsize'] = [12, 8]

# Define folder and filename of data

In [ ]:
folder = './'

files = [i for i in os.listdir(folder) if i.endswith("_convolved_STEM_image.hdf5")]
images = [os.path.join(folder, i) for i in files]

images

# Loop through simulated STEM reference images and rotate

In [ ]:
for image in images:

    # Read simulated STEM image from hdf5 file
    file = h5py.File(image,'r')
    img = file.get('convolved_stem_image/conv_stem')
    pixel_size = [pixel_to_angstrom, pixel_to_angstrom] # Units: Angstrom. This needs to be adjusted if other resolutions are used
    img = np.array(img)
    
    # Original image size
    Nx, Ny = img.shape
    
    # If image size is odd, make even in the corresponding direction by removing a one pixel boundary
    if (Nx % 2) == 0:
        Nx = Nx
    else:
        img = img[:Nx-1,:]
        
    if (Ny % 2) == 0:
        Ny = Ny
    else:
        img = img[:,:Ny-1]
        
    Nx, Ny = img.shape
        
    # Specify the minimum and maximum rotation angles and the rotation increment
    rot_min = 0
    rot_max = 90
    rot_inc = 5
    rot_range = np.arange(rot_min,rot_max+rot_inc,rot_inc)
    
    # Specify the field of view of the rotated images in Angstrom, so that all images are cropped to that size
    fov = 12 # field of view in Angstrom

    # Prepare output array
    size_crop = int(np.round(fov/(np.round(pixel_size[0]*100)/100)))
    img_rotated = np.zeros((size_crop, size_crop, rot_range.size), dtype=float)
    Nx_rot, Ny_rot, Nz_rot = np.shape(img_rotated)

    for a0 in range(0, rot_range.size, 1):

        #Rotate images
        img_rotated_temp = ndimage.rotate(img, rot_range[a0], reshape=False)

        #Crop image to maintain same field of view of here 1.2x1.2nm**2, if rectangular use smaller dimension
        #if (Nx-Ny) < 0:
        #    crop = Nxrep-Nx, Nyrep-Ny
        #elif (Nx-Ny) > 0:
        #    crop = int(np.round(((Nyrot*pixel_size[0]-fov)/pixel_size[0])/2))
        #else:
        #    crop = int(np.round(((Nxrot*pixel_size[0]-fov)/pixel_size[0])/2))
        
        crop = int((Nx-Nx_rot)/2), int((Ny-Ny_rot)/2)

        #Crop image
        img_rotated[:,:,a0] = img_rotated_temp[crop[0]:int(Nx-crop[0]),crop[1]:int(Ny-crop[1])]
    
    # Close file before writing new content
    file.close()
    
    # Save to hdf5 file
    with h5py.File(image[:-5] + '_augmented_images_rotation.hdf5', 'w') as f_rotation:
        group_image_rotation = f_rotation.create_group('Image_rotation')
        group_image_rotation.create_dataset('Rotated_images', data=img_rotated)
        group_image_rotation.create_dataset('Rotation_angles', data=rot_range)
        
        # Create attributes
        group_image_rotation.attrs['Data description'] = ['Each simulated STEM image is roated by the rotation angles specified in Rotation_angles and is cropped to a field of view of 1.2nm x 1.2nm (here 100x100 pixels)']

# Loop through simulated and rotated STEM images and apply shear transformation

### Shear of the images is related to linear sample drift

In [ ]:
from skimage.transform import warp, AffineTransform

for image in images:

    # Read simulated STEM image from hdf5 file
    file = h5py.File(image,'r')
    img = file.get('convolved_stem_image/conv_stem')
    pixel_size = [pixel_to_angstrom, pixel_to_angstrom] # Units: Angstrom. This needs to be adjusted if other resolutions are used.
    img = np.array(img)
    
    # Original image size
    Nx, Ny = img.shape
    
    # If image size is odd, make even in the corresponding direction by removing a one pixel boundary
    if (Nx % 2) == 0:
        Nx = Nx
    else:
        img = img[:Nx-1,:]
        
    if (Ny % 2) == 0:
        Ny = Ny
    else:
        img = img[:,:Ny-1]
        
    Nx, Ny = img.shape
        
    # Specify the minimum and maximum rotation angles and the rotation increment
    rot_min = 0
    rot_max = 90
    rot_inc = 5
    rot_range = np.arange(rot_min,rot_max+rot_inc,rot_inc)
    
    # Specify the minimum and maximum shear to be applied
    shear_min = -0.05
    shear_max = 0.05
    shear_inc = 0.01
    shear_range = np.arange(shear_min,shear_max+shear_inc/2,shear_inc)
    shear_range = np.round(shear_range, decimals=2)
    
    # Specify the field of view of the rotated images in Angstrom, so that all images are cropped to that size
    fov = 12 # field of view in Angstrom

    # Prepare output array
    size_crop = int(np.round(fov/(np.round(pixel_size[0]*100)/100)))
    img_rotated_sheared = np.zeros((size_crop, size_crop, rot_range.size, shear_range.size), dtype=float)
    Nx_rot_shear, Ny_rot_shear, Nz_rot_shear, Ns_rot_shear = np.shape(img_rotated_sheared)
    
    ## Set up temporary array for rotated and sheared images
    #img_rotated_sheared_temp = np.zeros((Nx_rep, Ny_rep, shear_range.size), dtype=float)

    for a0 in range(0, rot_range.size, 1):

        #Rotate images
        img_rotated_temp = ndimage.rotate(img, rot_range[a0], reshape=False)

        #Crop image to maintain same field of view of here 1.2x1.2nm**2, if rectangular use smaller dimension
        #if (Nx-Ny) < 0:
        #    crop = Nxrep-Nx, Nyrep-Ny
        #elif (Nx-Ny) > 0:
        #    crop = int(np.round(((Nyrot*pixel_size[0]-fov)/pixel_size[0])/2))
        #else:
        #    crop = int(np.round(((Nxrot*pixel_size[0]-fov)/pixel_size[0])/2))
        
        for b0 in range(0, shear_range.size, 1):
            
            tform = AffineTransform(scale=(1.0, 1.0), rotation=0, shear=shear_range[b0],
                        translation=(0, 0))
            img_rotated_sheared_temp = warp(img_rotated_temp, tform.inverse, output_shape=(Nx, Ny))

            #Crop image and write to array
            crop = int((Nx-Nx_rot)/2), int((Ny-Ny_rot)/2)
            img_rotated_sheared[:,:,a0,b0] = img_rotated_sheared_temp[crop[0]:int(Nx-crop[0]),crop[1]:int(Ny-crop[1])]
            
    # Close hdf5 file(s)  
    file.close()
    
    # Save to hdf5 file
    with h5py.File(image[:-5] + '_augmented_images_rotation&shear.hdf5', 'a') as f_rotation_shear:
        group_image_rotation_shear = f_rotation_shear.create_group('Image_rotation_and_shear')
        group_image_rotation_shear.create_dataset('Rotated_and_sheared_images', data=img_rotated_sheared)
        group_image_rotation_shear.create_dataset('Rotation_angles', data=rot_range)
        group_image_rotation_shear.create_dataset('Shear_range',data=shear_range)
        
        # Create attributes
        group_image_rotation_shear.attrs['Data description'] = ['Rotated_and_sheared_images dimensions: [0: image size x; 1: image size y; 2: rotation angles (rot_range); 3: shear level (shear_range)]']

# Test selected augmented images

In [ ]:
# Select file 'f_sel' and dataset 'data_sel'
file_sel = 2
data_sel = 2

with h5py.File(images[file_sel][:-5] + '_augmented_images_rotation&shear.hdf5', 'r') as f:
    
    # Print name of data file
    print(images[file_sel][:-5])
    
    # Extract datasets for each group
    file_data = f.get('Image_rotation_and_shear')
    img_data = file_data.get('Rotated_and_sheared_images')
    print(img_data)
    
    plt.imshow(img_data[:,:,0,data_sel])
    plt.axis('off')
    plt.colorbar()
    plt.show()